In [64]:
import pandas as pd
import numpy as np

import seaborn as sns
import warnings

# Funcoes auxiliares
from sklearn.preprocessing import OneHotEncoder


# Modelos de Aprendizado Nao-Supervisionado
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from sklearn.metrics import silhouette_score,rand_score,adjusted_rand_score

warnings.filterwarnings("ignore")

In [65]:
df = pd.read_csv('https://raw.githubusercontent.com/ImagineDogs/TranstornosMentais/main/dados/df_cid10.csv')
table = df[['cid10_diagnostico', 'cid10_seg_diagnostico', 'cid10_faixa', 'cid10_faixa_alta', 'cid10_seg_faixa', 'cid10_seg_faixa_alta']].loc[~df['cid10_faixa'].isnull()]
table = table.fillna('Sem')

In [66]:
encX = OneHotEncoder()

X = table[['cid10_diagnostico', 'cid10_seg_diagnostico', 'cid10_faixa', 'cid10_seg_faixa']]
X = encX.fit_transform(X)
y1 = np.array(table['cid10_faixa_alta'])
y2 = np.array(table['cid10_seg_faixa_alta'])

In [67]:

dbscan = DBSCAN()

clusters = dbscan.fit_predict(X)

silhouette = silhouette_score(X, clusters)
rand = rand_score(y1, clusters)
adj_rand = adjusted_rand_score(y1, clusters)

# Imprimindo as métricas
print("Silhueta:", silhouette)
print("Índice Rand:", rand)
print("Rand Corrigido:", adj_rand)

Silhueta: 0.8811144106014954
Índice Rand: 0.7620371231750308
Rand Corrigido: 0.14868541255647683


In [68]:

dbscan = DBSCAN()

clusters = dbscan.fit_predict(X)

silhouette = silhouette_score(X, clusters)
rand = rand_score(y2, clusters)
adj_rand = adjusted_rand_score(y2, clusters)

# Imprimindo as métricas
print("Silhueta:", silhouette)
print("Índice Rand:", rand)
print("Rand Corrigido:", adj_rand)

Silhueta: 0.8811144106014954
Índice Rand: 0.5536266765921329
Rand Corrigido: 0.028472532877529922


In [69]:
n_clusters= 11

kmeans = KMeans(n_clusters=n_clusters)

# Ajuste o modelo K-Means aos dados X
clusters = kmeans.fit_predict(X)

# Calcule as métricas de avaliação
silhouette = silhouette_score(X, clusters)
rand = rand_score(y1, clusters)
adj_rand = adjusted_rand_score(y1, clusters)

# Imprima as métricas
print("Silhueta:", silhouette)
print("Índice Rand:", rand)
print("Rand Corrigido:", adj_rand)

Silhueta: 0.4224601105430743
Índice Rand: 0.7561151040143672
Rand Corrigido: 0.2013132747125377


In [70]:
n_clusters= 11

kmeans = KMeans(n_clusters=n_clusters)

# Ajuste o modelo K-Means aos dados X
clusters = kmeans.fit_predict(X)

# Calcule as métricas de avaliação
silhouette = silhouette_score(X, clusters)
rand = rand_score(y2, clusters)
adj_rand = adjusted_rand_score(y2, clusters)

# Imprima as métricas
print("Silhueta:", silhouette)
print("Índice Rand:", rand)
print("Rand Corrigido:", adj_rand)

Silhueta: 0.4033818093764329
Índice Rand: 0.5580363890127648
Rand Corrigido: 0.0514647958159794


In [71]:
# cid_alta = df.filter(regex=r'^cid10_.*_alta$').columns.tolist()
# print(cid_alta)

# descricao_alta = df.filter(regex=r'^descricao_.*_alta$').columns.tolist()
# print(descricao_alta)

# alta = cid_alta + descricao_alta
# df_alta = df[alta]